In [ ]:
## Testing
print("hello")

hello


In [8]:
## Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data
raw_df = pd.read_parquet("../data/raw/online_retail.parquet",engine="fastparquet")


In [ ]:
## Clean the raw data

In [ ]:
## Exploratory Data Analysis - DO NOT INCLUDE IN PIPELINE
raw_df.head() ## 500K rows, which are line items for invoices.
raw_df.describe() ## Negative quantity and price metrics, likely refunds. Should be removed.
(raw_df["Price"] < 0).sum() ## 3 line items are negative, so will remove these.
raw_df.info() ## There are about 100K line items that are null.
raw_df.isnull().sum() ## Nulls are all in customer ID feature.
raw_df.shape # Shape before cleaning.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  525461 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


(525461, 8)

In [ ]:
## Full dataset cleaning
full_df = raw_df[(raw_df["Price"] >= 0) & (raw_df["Quantity"] >= 0)].dropna(subset=["Customer ID"])
full_df.shape # Shape after cleaning. Looks like there were an additional 10K rows removed for negative price and quantity.

(407695, 8)

In [31]:
full_df.head(20)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [32]:
## Group into customer-level dataframe

## First create lineitem_amount column
full_df["lineitem_amount"] = full_df["Quantity"] * full_df["Price"]
full_df.head(20)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,lineitem_amount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom,39.60
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.50
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.60
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom,45.00


In [ ]:
## Get max date within the entire dataset as reference for 'today'
MAX_DATE = full_df["InvoiceDate"].max()
print(MAX_DATE)

2010-12-09 20:01:00


In [34]:
## Create invoice df
invoice_df = (
    full_df
    .groupby(["Customer ID", "Invoice"], as_index=False)
    .agg({
        "lineitem_amount": "sum",       # Sum monetary value per invoice
        "InvoiceDate": "first" # Keep invoice date
    })
)

In [ ]:
## Check on results
invoice_df["Invoice"].str.startswith("C").describe() 
invoice_df["Invoice"].str.startswith("c").describe() ## According to this, no invoices were cancelled



count     19215
unique        1
top       False
freq      19215
Name: Invoice, dtype: object

In [ ]:
## 